In [1]:
import torch
import sigkernel2
import sigkernel
device = torch.cuda.device('cuda')

In [2]:
def generate(batch_size, length, dimension, device = torch.device('cpu')):
  random_walks = torch.randn(batch_size, length, dimension, dtype = torch.double, device = device)
  random_walks = torch.cumsum(random_walks, dim=1)
  return random_walks

In [3]:
sig = sigkernel2.SigKernel(sigkernel2.RBFKernel(1), 0)
sig1 = sigkernel.SigKernel(sigkernel.RBFKernel(1), 0)

In [4]:
X = generate(1, 4, 7, device = torch.device('cuda:0'))
Y = generate(1, 4, 7, device = torch.device('cuda:0'))

In [5]:
print(sig.compute_kernel(X, X))
print(sig1.compute_kernel(X, X))

tensor([[[ 1.9984, -0.9515, -0.0477],
         [-0.9515,  1.9044, -0.9522],
         [-0.0477, -0.9522,  1.9999]]], device='cuda:0', dtype=torch.float64)


/home/paperspace/.local/lib/python3.9/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


tensor([10.6187], device='cuda:0', dtype=torch.float64)


In [6]:
test = sig.compute_kernel(X, X)

In [15]:
test[:,:-1,:-1] + test[:,1:,1:] + test[:,1:,:-1] - test[:,:-1,1:]


tensor([[[ 2.0000e+00,  1.2393e+00,  1.7805e-03],
         [-7.6016e-01,  2.0000e+00,  1.0015e+00],
         [-1.2341e-03, -9.9849e-01,  2.0000e+00]]], device='cuda:0',
       dtype=torch.float64)

In [12]:
test

tensor([[[1.0000e+00, 2.3806e-01, 2.7322e-04, 8.0484e-14],
         [2.3806e-01, 1.0000e+00, 1.5073e-03, 5.2911e-12],
         [2.7322e-04, 1.5073e-03, 1.0000e+00, 5.8249e-07],
         [8.0484e-14, 5.2911e-12, 5.8249e-07, 1.0000e+00]]], device='cuda:0',
       dtype=torch.float64)